In [1]:
%matplotlib inline

import os
import datetime

import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf

mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
pd. set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 0)

In [3]:
ticker_list = ["aapl", "amzn", "msft"]

ticker = ticker_list[2]

ticker_upper = ticker.upper()

print(ticker_upper)

MSFT


In [4]:
alphavantage_csv_path = f"/app/StockPricePredictions/data/alphavantage/time_series_daily_adjusted/{ticker_upper}/{ticker_upper}.csv"

In [5]:
df = pd.read_csv(alphavantage_csv_path, low_memory=False)
df.sort_values(by=["date"], ascending=True, inplace=True)

df = df[df["date"]>='2010-01-01']

df["date_time"] = pd.to_datetime(df['date'])

df["day_of_week"] = df["date_time"].dt.dayofweek

df["adjusted_close_shift"] = df["5. adjusted close"].shift(-1)

df["percentage_change"] = (df["adjusted_close_shift"] - df["5. adjusted close"]) / df["5. adjusted close"] * 100.0

df["increase"] = df["percentage_change"].apply(lambda x: 1 if x > 0 else 0)

df.set_index("date", inplace=True)

In [6]:
df.shape[0]

3064

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3064 entries, 2010-01-04 to 2022-03-04
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   1. open               3064 non-null   float64       
 1   2. high               3064 non-null   float64       
 2   3. low                3064 non-null   float64       
 3   4. close              3064 non-null   float64       
 4   5. adjusted close     3064 non-null   float64       
 5   6. volume             3064 non-null   float64       
 6   7. dividend amount    3064 non-null   float64       
 7   8. split coefficient  3064 non-null   float64       
 8   date_time             3064 non-null   datetime64[ns]
 9   day_of_week           3064 non-null   int64         
 10  adjusted_close_shift  3063 non-null   float64       
 11  percentage_change     3063 non-null   float64       
 12  increase              3064 non-null   int64         
dtypes: datet

In [8]:
df.tail()
# df[df.percentage_change == 0]

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,date_time,day_of_week,adjusted_close_shift,percentage_change,increase
date,,,,,,,,,,,,,
2022-02-28,294.31,299.14,293.000,298.79,298.79,34627457.0,0.0,1.0,2022-02-28,0,294.95,-1.285184,0
2022-03-01,296.40,299.97,292.150,294.95,294.95,31217778.0,0.0,1.0,2022-03-01,1,300.19,1.776572,1
2022-03-02,295.36,301.47,293.698,300.19,300.19,31873007.0,0.0,1.0,2022-03-02,2,295.92,-1.422432,0
2022-03-03,302.89,303.13,294.050,295.92,295.92,27314469.0,0.0,1.0,2022-03-03,3,289.86,-2.047851,0
2022-03-04,294.29,295.66,287.165,289.86,289.86,32369655.0,0.0,1.0,2022-03-04,4,NaN,NaN,0


In [9]:
df.increase.value_counts()

1    1608
0    1456
Name: increase, dtype: int64

In [10]:
# The day of the week with Monday=0, Sunday=6.

# 5 = Saturday
# 6 = Sunday
df.day_of_week.value_counts()

1    629
2    628
3    619
4    614
0    574
Name: day_of_week, dtype: int64

In [11]:
loc = df.index.get_loc('2010-01-05')
loc

print(loc)

df.iloc[loc]["date_time"].strftime("%Y-%m-%d")

1


'2010-01-05'

In [12]:
benzinga_csv_path = f"/app/StockPricePredictions/data/alphavantage/time_series_daily_adjusted/{ticker_upper}/{ticker}_finbert_20100101_20220304.csv"

In [13]:
df_benzinga = pd.read_csv(benzinga_csv_path, low_memory=False)

In [14]:
df_benzinga['date'] = pd.to_datetime(df_benzinga['Stock'])

In [15]:
df_benzinga["day_of_week"] = df_benzinga["date"].dt.dayofweek

In [16]:
df_benzinga.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11106 entries, 0 to 11105
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Headline     11106 non-null  object        
 1   Stock        11106 non-null  object        
 2   Positive     11106 non-null  float64       
 3   Negative     11106 non-null  float64       
 4   Neutral      11106 non-null  float64       
 5   date         11106 non-null  datetime64[ns]
 6   day_of_week  11106 non-null  int64         
dtypes: datetime64[ns](1), float64(3), int64(1), object(2)
memory usage: 607.5+ KB


In [17]:
df_benzinga.head(1)

,Headline,Stock,Positive,Negative,Neutral,date,day_of_week
0,"Stock Market News for January 04, 2010 - Market News U.S. stocks fell broadly on the last trading day of an otherwise great year for equities as a better-than-estimated report on initial jobless claims stoked investors’ concern that the Federal Reserve would soon begin winding up stimulus measures. The Dow Jones industrial average lost 120.46 points, or 1.1%, to 10,428.05 as 29 of its 30 components ended in the red in a thinly traded session. The broader S&P 500 index slid more than 11 points, or 1%, to 1,115.1 points, narrowing its 2009 gain to 23%. The tech-laden Nasdaq closed down 22.13 points, or about 1%, at 2,269.15. This morning’s stock futures suggest Wall Street is headed for a higher opening on 2010’s first trading day. Dow Jones industrial average futures are up 58 points, or 0.6%, to 10,423. Standard & Poor's 500 index futures are up 6.60 points, or 0.6%, to 1,117.30, while Nasdaq 100 index futures are up 21.75, or 1.2%, to 1,880.50. During the year, the S&P 500 gained 23.4%, fed by recovery hopes. The DJIA finished the year up 18.8%, or 1652 points, for its best run since 2003, led by a flamboyant 118.4% jump over the year in American Express , a 56.8% surge in Microsoft , and a 55.5% gain in IBM . Nasdaq, helped by strength in its technology components, jumped 44% and recorded a 6.9% fourth-quarter gain. Technology stocks were the leading gainers among the S&P 500 industry sectors, surging 60% as a group, followed by a 45% rise in raw-materials producers and an almost 40% surge in consumer-related companies. The upcoming week witnesses increased activity on the Fed front, with the Wednesday FOMC minutes commanding the most attention. Yesterday Lockhart, Kohn and Bernanke took to the podium; today Lockhart and Duke are slated to speak; on Tuesday Hoenig would take to the podium; Bullard is slated on Friday. Of note is the recent advance in Treasury yields, as the US government funds the rising deficit. The 10-year yield has climbed about 65 basis points in December to 3.85%, rising to a multi-month high of 3.91% Thursday, as investors grew concerned that Fed would slowly raise interest rate due to the improved economic data, which in turn engendered less safe-haven buying.Zacks Investment Research",2010-01-04,0.208157,0.760072,0.031771,2010-01-04,0


In [18]:
df_benzinga.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11106 entries, 0 to 11105
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Headline     11106 non-null  object        
 1   Stock        11106 non-null  object        
 2   Positive     11106 non-null  float64       
 3   Negative     11106 non-null  float64       
 4   Neutral      11106 non-null  float64       
 5   date         11106 non-null  datetime64[ns]
 6   day_of_week  11106 non-null  int64         
dtypes: datetime64[ns](1), float64(3), int64(1), object(2)
memory usage: 607.5+ KB


In [19]:
# The day of the week with Monday=0, Sunday=6.

# 5 = Saturday
# 6 = Sunday

df_benzinga.day_of_week.value_counts()

2    2278
3    2237
1    2222
0    1984
4    1887
6    276 
5    222 
Name: day_of_week, dtype: int64

In [20]:
df_benzinga.head(1)

,Headline,Stock,Positive,Negative,Neutral,date,day_of_week
0,"Stock Market News for January 04, 2010 - Market News U.S. stocks fell broadly on the last trading day of an otherwise great year for equities as a better-than-estimated report on initial jobless claims stoked investors’ concern that the Federal Reserve would soon begin winding up stimulus measures. The Dow Jones industrial average lost 120.46 points, or 1.1%, to 10,428.05 as 29 of its 30 components ended in the red in a thinly traded session. The broader S&P 500 index slid more than 11 points, or 1%, to 1,115.1 points, narrowing its 2009 gain to 23%. The tech-laden Nasdaq closed down 22.13 points, or about 1%, at 2,269.15. This morning’s stock futures suggest Wall Street is headed for a higher opening on 2010’s first trading day. Dow Jones industrial average futures are up 58 points, or 0.6%, to 10,423. Standard & Poor's 500 index futures are up 6.60 points, or 0.6%, to 1,117.30, while Nasdaq 100 index futures are up 21.75, or 1.2%, to 1,880.50. During the year, the S&P 500 gained 23.4%, fed by recovery hopes. The DJIA finished the year up 18.8%, or 1652 points, for its best run since 2003, led by a flamboyant 118.4% jump over the year in American Express , a 56.8% surge in Microsoft , and a 55.5% gain in IBM . Nasdaq, helped by strength in its technology components, jumped 44% and recorded a 6.9% fourth-quarter gain. Technology stocks were the leading gainers among the S&P 500 industry sectors, surging 60% as a group, followed by a 45% rise in raw-materials producers and an almost 40% surge in consumer-related companies. The upcoming week witnesses increased activity on the Fed front, with the Wednesday FOMC minutes commanding the most attention. Yesterday Lockhart, Kohn and Bernanke took to the podium; today Lockhart and Duke are slated to speak; on Tuesday Hoenig would take to the podium; Bullard is slated on Friday. Of note is the recent advance in Treasury yields, as the US government funds the rising deficit. The 10-year yield has climbed about 65 basis points in December to 3.85%, rising to a multi-month high of 3.91% Thursday, as investors grew concerned that Fed would slowly raise interest rate due to the improved economic data, which in turn engendered less safe-haven buying.Zacks Investment Research",2010-01-04,0.208157,0.760072,0.031771,2010-01-04,0


In [21]:
df_benzinga["in_index"] = df_benzinga["Stock"].apply(lambda x: True if x in df.index else False)

In [22]:
df_benzinga[df_benzinga.in_index==False]["day_of_week"].value_counts()

6    276
5    222
0    39 
4    7  
1    3  
3    2  
2    1  
Name: day_of_week, dtype: int64

In [23]:
df.index.get_loc('2010-01-04')


0

In [24]:
from datetime import date, datetime, timedelta

In [25]:
test2 =  '2010-01-09'

dtobj1 = datetime.strptime(test2, "%Y-%m-%d")

print(dtobj1)

days = timedelta(2)

print(days)



dtobj2 = (dtobj1 - days).strftime("%Y-%m-%d")
dtobj2

2010-01-09 00:00:00
2 days, 0:00:00


'2010-01-07'

In [26]:
def update_date(x):

    global ERROR_COUNTER

    if x.in_index == True:
        return x.Stock
    else:
        # try:

            dt_time = datetime.strptime(x.Stock, "%Y-%m-%d")

            # loc = df.index.get_loc(date_part)
            if x.day_of_week == 6: # Sunday to Thursday
                days = timedelta(3)
                idx_lookup = (dt_time - days).strftime("%Y-%m-%d")
                print(idx_lookup)
                try:
                    loc = df.index.get_loc(idx_lookup)
                    return df.iloc[loc]["date_time"].strftime("%Y-%m-%d")
                except Exception as e1:
                    ERROR_COUNTER += 1
                    print(str(e1))
                    return x.Stock
            if x.day_of_week == 5: # Saturday to Thursday
                days = timedelta(2)
                idx_lookup = (dt_time - days).strftime("%Y-%m-%d")
                print(idx_lookup)
                try:
                    loc = df.index.get_loc(idx_lookup)
                    return df.iloc[loc]["date_time"].strftime("%Y-%m-%d")
                except Exception as e2:
                    ERROR_COUNTER += 1
                    print(str(e2))
                    return x.Stock
            if x.day_of_week == 4: # Friday to Thursday
                days = timedelta(1)
                idx_lookup = (dt_time - days).strftime("%Y-%m-%d")
                print(idx_lookup)
                try:
                    loc = df.index.get_loc(idx_lookup)
                    return df.iloc[loc]["date_time"].strftime("%Y-%m-%d")
                except Exception as e3:
                    ERROR_COUNTER += 1
                    print(str(3))
                    return x.Stock
            if x.day_of_week == 0: # Monday to Thursday
                days = timedelta(4)
                idx_lookup = (dt_time - days).strftime("%Y-%m-%d")
                print(idx_lookup)
                try:
                    loc = df.index.get_loc(idx_lookup)
                    return df.iloc[loc]["date_time"].strftime("%Y-%m-%d")
                except Exception as e3:
                    ERROR_COUNTER += 1
                    print(str(3))
                    return x.Stock

            else:
                return x.Stock

        # except Exception as e:
        #     ERROR_COUNTER += 1
        #     print(str(e))
        #     print(x.day_of_week, x.Stock, x.in_index)
        #     return x.Stock
    # loc = df.index.get_loc('2010-01-01')
    # loc



In [27]:
# The day of the week with Monday=0, Sunday=6.

# 5 = Saturday
# 6 = Sunday

ERROR_COUNTER = 0 

df_benzinga["backfill_date"] = df_benzinga["Stock"] # df_benzinga.apply(update_date, axis=1)


In [28]:
ERROR_COUNTER

0

In [29]:
df_benzinga["in_index"] = df_benzinga["backfill_date"].apply(lambda x: True if x in df.index else False)

In [30]:
df_benzinga[df_benzinga.in_index==False]["day_of_week"].value_counts()

6    276
5    222
0    39 
4    7  
1    3  
3    2  
2    1  
Name: day_of_week, dtype: int64

In [31]:
df_benzinga[df_benzinga.in_index==False].shape

(550, 9)

In [32]:
# df_benzinga["text"] = df_benzinga["title"] + " " + df_benzinga["body"]

In [33]:
# df_benzinga['text'] = np.where(df_benzinga["text"], df_benzinga["title"], df_benzinga["text"])

In [34]:
df_benzinga = df_benzinga[["backfill_date", "Positive", "Negative", "Neutral"]]

In [35]:
df_benzinga.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11106 entries, 0 to 11105
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   backfill_date  11106 non-null  object 
 1   Positive       11106 non-null  float64
 2   Negative       11106 non-null  float64
 3   Neutral        11106 non-null  float64
dtypes: float64(3), object(1)
memory usage: 347.2+ KB


In [36]:
df_benzinga.head()

,backfill_date,Positive,Negative,Neutral
0,2010-01-04,0.208157,0.760072,0.031771
1,2010-01-04,0.024145,0.808129,0.167726
2,2010-01-04,0.922880,0.023758,0.053362
3,2010-01-04,0.143269,0.012789,0.843941
4,2010-01-05,0.874124,0.063757,0.062120


In [37]:
df_benzinga.rename(columns={"backfill_date": "date"}, inplace=True)

In [38]:
def clean_tweet(df):
    # replace URLs with a whitespace
    df['text'] = df['text'].str.replace('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', ' ')
    
    return df

In [39]:
df_benzinga.head()

,date,Positive,Negative,Neutral
0,2010-01-04,0.208157,0.760072,0.031771
1,2010-01-04,0.024145,0.808129,0.167726
2,2010-01-04,0.922880,0.023758,0.053362
3,2010-01-04,0.143269,0.012789,0.843941
4,2010-01-05,0.874124,0.063757,0.062120


In [40]:
# df_benzinga = clean_tweet(df_benzinga)

In [41]:
df_benzinga.set_index("date", inplace=True)

In [42]:
df_benzinga.head()

,Positive,Negative,Neutral
date,,,
2010-01-04,0.208157,0.760072,0.031771
2010-01-04,0.024145,0.808129,0.167726
2010-01-04,0.922880,0.023758,0.053362
2010-01-04,0.143269,0.012789,0.843941
2010-01-05,0.874124,0.063757,0.062120


In [43]:
df_benzinga_duplicated_index = df_benzinga[df_benzinga.index.duplicated(keep=False)]

In [44]:
df_benzinga_duplicated_index.shape[0]

10228

In [45]:
df_benzinga_nonduplicated_index = df_benzinga[~df_benzinga.index.duplicated(keep=False)]

In [46]:
df_benzinga_nonduplicated_index.shape[0]

878

In [47]:
# df_twint.groupby('date')['negative'].mean()

df_benzinga_groupby = df_benzinga.groupby("date").agg(
     negative = ("Negative", "mean"),
     nuetral = ("Neutral", "mean"),
     positive = ("Positive", "mean"),
     # compound = ("compound", "mean"),
     # nlikes = ("nlikes", "sum"),
     # nretweets = ("nretweets", "sum"),
     )


In [48]:
df_benzinga_groupby

,negative,nuetral,positive
date,,,
2010-01-04,0.401187,0.274200,0.324613
2010-01-05,0.108230,0.176608,0.715163
2010-01-06,0.050018,0.339652,0.610330
2010-01-07,0.029928,0.195104,0.774968
2010-01-08,0.319698,0.402485,0.277817
...,...,...,...
2022-02-28,0.397671,0.531703,0.070626
2022-03-01,0.083546,0.596145,0.320308
2022-03-02,0.522847,0.434464,0.042689


In [49]:
df_merge = pd.merge(df, df_benzinga_groupby, how="inner", left_index=True, right_index=True)

In [50]:
df_merge.shape

(2682, 16)

In [51]:
df_merge.day_of_week.value_counts()

1    565
2    555
3    545
4    513
0    504
Name: day_of_week, dtype: int64

In [52]:
# df.index
# df_text.index
df_merge[df_merge.negative.isnull()]
# df_merge.shape

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,date_time,day_of_week,adjusted_close_shift,percentage_change,increase,negative,nuetral,positive
date,,,,,,,,,,,,,,,,


In [53]:
df.shape

(3064, 13)

In [54]:
df_benzinga_groupby.shape

(3105, 3)

In [55]:
# df_merge.fillna(0.0, inplace=True)
# df_merge.fillna(method="ffill", inplace=True)

In [56]:
df_merge[df_merge.negative.isnull()].shape

(0, 16)

In [57]:
df_merge.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2682 entries, 2010-01-04 to 2022-03-04
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   1. open               2682 non-null   float64       
 1   2. high               2682 non-null   float64       
 2   3. low                2682 non-null   float64       
 3   4. close              2682 non-null   float64       
 4   5. adjusted close     2682 non-null   float64       
 5   6. volume             2682 non-null   float64       
 6   7. dividend amount    2682 non-null   float64       
 7   8. split coefficient  2682 non-null   float64       
 8   date_time             2682 non-null   datetime64[ns]
 9   day_of_week           2682 non-null   int64         
 10  adjusted_close_shift  2681 non-null   float64       
 11  percentage_change     2681 non-null   float64       
 12  increase              2682 non-null   int64         
 13  negative

In [58]:
# df_merge.fillna(0.0, inplace=True)

In [59]:
##############df_merge.to_csv(f"/app/StockPricePredictions/data/alphavantage/time_series_daily_adjusted/{ticker_upper}/{ticker_upper}_WITH_BENZINGA_FINBERT_SA.csv")
df_merge.head()

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,date_time,day_of_week,adjusted_close_shift,percentage_change,increase,negative,nuetral,positive
date,,,,,,,,,,,,,,,,
2010-01-04,30.62,31.10,30.59,30.950,23.857347,38409100.0,0.0,1.0,2010-01-04,0,23.865055,0.032310,1,0.401187,0.274200,0.324613
2010-01-05,30.85,31.10,30.64,30.960,23.865055,49749600.0,0.0,1.0,2010-01-05,1,23.718596,-0.613695,0,0.108230,0.176608,0.715163
2010-01-06,30.88,31.08,30.52,30.770,23.718596,58182400.0,0.0,1.0,2010-01-06,2,23.473471,-1.033474,0,0.050018,0.339652,0.610330
2010-01-07,30.63,30.70,30.19,30.452,23.473471,50559700.0,0.0,1.0,2010-01-07,3,23.633805,0.683042,1,0.029928,0.195104,0.774968
2010-01-08,30.28,30.88,30.24,30.660,23.633805,51197400.0,0.0,1.0,2010-01-08,4,23.333179,-1.272016,0,0.319698,0.402485,0.277817


In [60]:
# df_merge = df_merge.sample(frac=1).reset_index(drop=True)

In [61]:
df_merge.shape

(2682, 16)

In [62]:
# df_merge[(df_merge["negative"]==0.0) & (df_merge["nuetral"]==0.0) & (df_merge["positive"]==0.0)].shape

In [63]:
# df_merge = df_merge[~(df_merge["negative"]==0.0) & ~(df_merge["nuetral"]==0.0) & ~(df_merge["positive"]==0.0)].copy()

In [64]:
X = df_merge[["negative", "nuetral", "positive"]]
y = df_merge["increase"]

In [65]:
y.head()

date
2010-01-04    1
2010-01-05    0
2010-01-06    0
2010-01-07    1
2010-01-08    0
Name: increase, dtype: int64

In [66]:
SPLIT = int(0.8 * len(df_merge))

X_train = X[:SPLIT]
X_test = X[SPLIT:]

y_train = y[:SPLIT]
y_test = y[SPLIT:]



In [67]:
X_train

,negative,nuetral,positive
date,,,
2010-01-04,0.401187,0.274200,0.324613
2010-01-05,0.108230,0.176608,0.715163
2010-01-06,0.050018,0.339652,0.610330
2010-01-07,0.029928,0.195104,0.774968
2010-01-08,0.319698,0.402485,0.277817
...,...,...,...
2019-12-09,0.255585,0.556354,0.188062
2019-12-10,0.105180,0.348350,0.546470
2019-12-11,0.968667,0.019348,0.011984


In [68]:
from sklearn.ensemble import RandomForestClassifier

In [69]:
rfc = RandomForestClassifier(n_estimators=200, criterion="entropy")
rfc.fit(X_train, y_train)

RandomForestClassifier(criterion='entropy', n_estimators=200)

In [70]:
preds = rfc.predict(X_test)

In [71]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [72]:
matrix = confusion_matrix(y_test, preds)

In [73]:
matrix

array([[117, 131],
       [133, 156]])

In [74]:
score = accuracy_score(y_test, preds)

In [75]:
score

0.5083798882681564

In [76]:
print(classification_report(y_test, preds))


              precision    recall  f1-score   support

           0       0.47      0.47      0.47       248
           1       0.54      0.54      0.54       289

    accuracy                           0.51       537
   macro avg       0.51      0.51      0.51       537
weighted avg       0.51      0.51      0.51       537



In [77]:
corr_matrix = df_merge[["negative", "nuetral", "positive", "increase"]].corr()
print (corr_matrix)

          negative   nuetral  positive  increase
negative  1.000000 -0.622238 -0.417345  0.028612
nuetral  -0.622238  1.000000 -0.451706 -0.015453
positive -0.417345 -0.451706  1.000000 -0.014669
increase  0.028612 -0.015453 -0.014669  1.000000


In [78]:
from statsmodels.tsa.stattools import grangercausalitytests

In [79]:
#perform Granger-Causality test
grangercausalitytests(df_merge[["positive", "increase"]], maxlag=[1])


Granger Causality
number of lags (no zero) 1
ssr based F test:         F=17.6963 , p=0.0000  , df_denom=2678, df_num=1
ssr based chi2 test:   chi2=17.7161 , p=0.0000  , df=1
likelihood ratio test: chi2=17.6579 , p=0.0000  , df=1
parameter F test:         F=17.6963 , p=0.0000  , df_denom=2678, df_num=1


{1: ({'ssr_ftest': (17.69632311036183, 2.6764302447569563e-05, 2678.0, 1),
   'ssr_chi2test': (17.716147221389125, 2.564405310415813e-05, 1),
   'lrtest': (17.65786933720483, 2.644189870165294e-05, 1),
   'params_ftest': (17.696323110361906, 2.6764302447569563e-05, 2678.0, 1.0)},
   array([[0., 1., 0.]])])}